In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# Sample weekly data
df = pd.read_csv('NVDA_data.csv')
df.head()

,Year,Week_Number,Open,Adj Close,mean_return,volatility,color_label
0,2018,0,48.945000,53.287727,1.98875,3.081352,red
1,2018,1,55.099998,55.162941,0.70180,1.397675,red
2,2018,2,56.012501,56.926830,0.80200,1.813348,green
3,2018,3,57.607498,60.197319,1.13480,1.691356,green
4,2018,4,60.685001,57.770416,-0.80300,2.021694,red


In [3]:
train_data = df[df['Year'] != 2022].copy()
test_data = df[df['Year'] == 2022].copy()

X_train = train_data[['Open', 'Adj Close', 'mean_return', 'volatility']]
y_train = train_data['color_label']

X_test = test_data[['Open', 'Adj Close', 'mean_return', 'volatility']]
y_test_true = test_data['color_label']


In [4]:
# Linear SVM and its accuracy for Year 2
linear_svm = SVC(kernel='linear')
linear_svm.fit(X_train, y_train)
y_test_pred_linear = linear_svm.predict(X_test)
accuracy_linear_svm = accuracy_score(y_test_true, y_test_pred_linear)
print(f'Accuracy of Linear SVM for Year 2: {accuracy_linear_svm}')

#confusion matrix for Linear SVM
conf_matrix_linear_svm = confusion_matrix(y_test_true, y_test_pred_linear)
print('Confusion Matrix for Linear SVM (Year 2):')
print(conf_matrix_linear_svm)

#sensitivity (recall) and specificity for Linear SVM
tn_linear, fp_linear, fn_linear, tp_linear = conf_matrix_linear_svm.ravel()
sensitivity_linear = tp_linear / (tp_linear + fn_linear)
specificity_linear = tn_linear / (tn_linear + fp_linear)
print(f'True Positive Rate (Sensitivity) for Linear SVM: {sensitivity_linear:.4f}')
print(f'True Negative Rate (Specificity) for Linear SVM: {specificity_linear:.4f}')


Accuracy of Linear SVM for Year 2: 0.8461538461538461
Confusion Matrix for Linear SVM (Year 2):
[[12  2]
 [ 6 32]]
True Positive Rate (Sensitivity) for Linear SVM: 0.8421
True Negative Rate (Specificity) for Linear SVM: 0.8571


In [5]:
#Gaussian SVM and its accuracy for Year 2
gaussian_svm = SVC(kernel='rbf')
gaussian_svm.fit(X_train, y_train)
y_test_pred_gaussian = gaussian_svm.predict(X_test)
accuracy_gaussian_svm = accuracy_score(y_test_true, y_test_pred_gaussian)
print(f'Accuracy of Gaussian SVM for Year 2: {accuracy_gaussian_svm}')

#Polynomial SVM (degree=2) and its accuracy for Year 2
poly_svm = SVC(kernel='poly', degree=2)
poly_svm.fit(X_train, y_train)
y_test_pred_poly = poly_svm.predict(X_test)
accuracy_poly_svm = accuracy_score(y_test_true, y_test_pred_poly)
print(f'Accuracy of Polynomial SVM (degree=2) for Year 2: {accuracy_poly_svm}')

Accuracy of Gaussian SVM for Year 2: 0.7307692307692307
Accuracy of Polynomial SVM (degree=2) for Year 2: 0.7884615384615384


In [6]:
test_data['predicted_label'] = y_test_pred_linear
test_data.head()

,Year,Week_Number,Open,Adj Close,mean_return,volatility,color_label,predicted_label
211,2022,1,298.149994,272.118683,-1.4652,3.573360,red,red
212,2022,2,265.809998,269.072632,-0.1934,2.768013,red,red
213,2022,3,262.600006,233.438583,-3.4885,0.320452,red,red
214,2022,4,223.300003,228.105469,-0.4078,3.646720,red,red
215,2022,5,231.820007,242.876404,1.3404,4.416611,green,green


# Trading strategy for 2022 to calculate the amount at the end of the year using predicted labels

In [7]:
def trading_strategy_predicted_logistic_reg(df, initial_balance=100):
    df = df.reset_index(drop=True)  # Reset the index to numeric values
    balance = initial_balance
    position = None
    balance_history = []

    for week in range(len(df)):  # Adjust the loop condition
        open_price = df.loc[week, 'Open']
        close_price = df.loc[week, 'Adj Close']

        if df.loc[week, 'predicted_label'] == 'green':
            if position is None:
                shares_to_buy = balance / open_price
                print(f"Week {week}: Buying {shares_to_buy:.2f} shares at ${open_price:.2f}")
                position = shares_to_buy
                balance = 0
                print(f"   Shares: {position:.2f}, Portfolio Value: ${balance:.2f}")

        elif df.loc[week, 'predicted_label'] == 'red':
            if position is not None:
                balance_from_selling = position * df.loc[week - 1, 'Adj Close']
                print(f"Week {week}: Selling {position:.2f} shares at ${df.loc[week - 1, 'Adj Close']:.2f}, "
                      f"Portfolio Value: ${balance_from_selling:.2f}")
                balance += balance_from_selling
                position = None
                print(f"   Portfolio Value: ${balance:.2f}")

        if position is None:
            portfolio_value = balance
        else:
            portfolio_value = position * df.loc[week - 1, 'Adj Close']

        balance_history.append(portfolio_value)
        print(f"Week {week}: Portfolio Value: ${portfolio_value:.2f}")

    return balance_history

resulting_balance_history_lr = trading_strategy_predicted_logistic_reg(test_data)
print("Balance History based on Modified Predicted Labels for 2022:", resulting_balance_history_lr[-1])


Week 0: Portfolio Value: $100.00
Week 1: Portfolio Value: $100.00
Week 2: Portfolio Value: $100.00
Week 3: Portfolio Value: $100.00
Week 4: Buying 0.43 shares at $231.82
   Shares: 0.43, Portfolio Value: $0.00
Week 4: Portfolio Value: $98.40
Week 5: Selling 0.43 shares at $242.88, Portfolio Value: $104.77
   Portfolio Value: $104.77
Week 5: Portfolio Value: $104.77
Week 6: Portfolio Value: $104.77
Week 7: Buying 0.45 shares at $230.35
   Shares: 0.45, Portfolio Value: $0.00
Week 7: Portfolio Value: $107.39
Week 8: Selling 0.45 shares at $241.26, Portfolio Value: $109.73
   Portfolio Value: $109.73
Week 8: Portfolio Value: $109.73
Week 9: Portfolio Value: $109.73
Week 10: Buying 0.50 shares at $218.69
   Shares: 0.50, Portfolio Value: $0.00
Week 10: Portfolio Value: $110.77
Week 11: Portfolio Value: $132.58
Week 12: Selling 0.50 shares at $276.61, Portfolio Value: $138.79
   Portfolio Value: $138.79
Week 12: Portfolio Value: $138.79
Week 13: Portfolio Value: $138.79
Week 14: Portfolio V